In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
crowd_path = base_dir + "crowd/"

Mounted at /content/drive/


In [2]:
from gensim.parsing.preprocessing import STOPWORDS
 
# Adding multiple words to gensim stoplist
new_stopword = STOPWORDS.union(set(["\'", ',', '?','-', '--', '.', '``', '`', ';', '!', '\'s', '\'t','(', ')','"','""','.-','-.', 'n\'t', '\'d', 'A', '\'re', '\'m','%','$',':','&','&&','\'\'']))

def remove_custom_stopwords(p):
  tok_without_sw = [word for word in p if not word.lower() in new_stopword]
  p[:] = tok_without_sw

In [3]:
custom_stopwords=True

In [4]:
import json

with open(crowd_path + "train_m_tree.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [5]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  19980 elementi


In [6]:
examples_iter = iter(examples_train)

In [7]:
open(path_corpus_data + 'types_counters_choi.json', 'w').close()

In [8]:
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords
if custom_stopwords:
   fp = open(path_corpus_data + 'corpus_data_entire_choi_custom.txt', "w+")  # Open a file
else:
  fp = open(path_corpus_data + 'corpus_data_entire_choi.txt', "w+")  # Open a file

restricted_set = {}
for i in tqdm(examples_iter):
  if (len(i['y_str'])>1):
    for k in range(0,len(i['y_str'])):
      phrase = []
      phrase = i['left_context_token'] + ("/" + i['y_str'][k].upper()).split() + i['right_context_token']
      
      if custom_stopwords:
        remove_custom_stopwords(phrase)
        fp.write(' '.join(phrase.copy())+'\n')
      else:
        fp.write(remove_stopwords(' '.join(phrase)) + '\n')

      if ("/" + i['y_str'][k].upper() not in restricted_set):
          restricted_set["/" + i['y_str'][k].upper()]=1
      else:
          restricted_set["/" + i['y_str'][k].upper()]+=1
  else:
    phrase = []
    phrase = i['left_context_token'] +  ("/" + i['y_str'][0].upper()).split() + i['right_context_token']
    
    if custom_stopwords:
      remove_custom_stopwords(phrase)
      fp.write(' '.join(phrase.copy())+'\n')
    else:
      fp.write(remove_stopwords(' '.join(phrase)) + '\n')

    if ("/" + i['y_str'][0].upper() not in restricted_set):
      restricted_set["/" + i['y_str'][0].upper()]=1
    else:
      restricted_set["/" + i['y_str'][0].upper()]+=1

fp.close()


19980it [00:01, 16648.47it/s]


In [9]:
  with open(path_corpus_data + 'types_counters_choi.json', 'a') as fp_3:
      json.dump(restricted_set, fp_3)